<a href="https://colab.research.google.com/github/purvi202211023/IT555-Deep-Neural-NLP/blob/Train-Word2Vec-on-peS2o-Dataset-(AllenNLP)-_Assignment1/Deep_Neural_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
**Team NeuronicNav**

Assignment 1: Train Word2Vec on peS2o Dataset (AllenNLP)

Id No.: 202211023, 202211004, 202221004

---




In [ ]:
!pip install num2words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import inflect
import pandas as pd
import numpy as np
from num2words import num2words  # For numerical-to-text conversion

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Load the training data
# df_train_n = pd.read_pickle("/content/drive/MyDrive/NLP/train_data_n.pkl")

# # Load the test data
# df_test_n = pd.read_pickle("/content/drive/MyDrive/NLP/test_data_n.pkl")

Q1: Research papers are collected only from Computer science domain


  I have collected 4000 documents from a dataset and then store in one pickle file that I load here for the further use. I'll use bunch of keyword for filters CS domain's paper

  
Q2: 2. A random training data of 3000 documents are used, while a test dataset of 1000 documents is to be used.

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Datasets/df_new.pkl")
df_train = df[:3000]
df_test = df[3000:4000]

In [ ]:
df

,id,source,version,added,created,text
0,79374620,s2ag/train,v1,2019-03-16T13:05:58.356Z,2006-01-01T00:00:00.000Z,In-time rating of continuously cast semis by m...
1,53596570,s2ag/train,v1,2018-11-07T20:43:16.101Z,2003-09-24T00:00:00.000Z,Development of an Inspection System for Cracks...
2,60680010,s2ag/train,v1,2019-02-13T14:02:18.109Z,2004-01-01T00:00:00.000Z,Assessment of Wave-Induced Liquefaction in a P...
3,254100620,s2ag/train,v1,2022-12-01T14:30:17.430Z,2022-10-10T00:00:00.000Z,Prediction of Cascading Failures and Simultane...
4,17912070,s2ag/train,v1,2014-10-01T00:00:00.000Z,2001-01-01T00:00:00.000Z,2 3 Ju l 2 00 1 A Logical Framework for Conver...
...,...,...,...,...,...,...
3995,29202040,s2ag/train,v1,2017-09-06T09:13:11.852Z,2016-01-01T00:00:00.000Z,Computational Social Science: Discovery and Pr...
3996,64350490,s2ag/train,v1,2019-02-16T14:30:11.842Z,2014-03-24T00:00:00.000Z,Toward Automated Interpretation of Integrated ...
3997,62537940,s2ag/train,v1,2019-02-14T14:17:03.140Z,1997-07-14T00:00:00.000Z,Application of virtual reality technology to c...
3998,7418240,s2ag/train,v1,2017-02-10T07:10:48.762Z,2004-11-02T00:00:00.000Z,Three level method using machine learning and ...


Q3: The linguistic unit (row vector) in your input context matrix should be sentences (and not words). That is, you are to embed sentences instead of words.

Q4: The following pre-processing cleanup needs to be done on the input context matrix: (i) stopwords, (ii) urls, (iii) bullets, (iv) apostrophe, (v) hyphens, (vi) enumerations (eg. like how this list is being enumerated), (vii), numerical-to-text conversion, (viii) punctuations.

In [ ]:
# Create an inflect engine for number-to-text conversion
p = inflect.engine()

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove bullets
    text = re.sub(r'\u2022', '', text)

    # Handle apostrophe
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)

    # Handle hyphens
    text = text.replace("-", " ")
    text = text.replace("_", " ")

    # Handle enumerations
    text = re.sub(r'\(i\)|\(ii\)|\(iii\)|\(iv\)|\(v\)|\(vi\)|\(vii\)|\(viii\)|\(ix\)|\(x\)', '', text)

    # Convert numbers to text
    text = ' '.join([p.number_to_words(word) if bool(re.search(r'\d', word)) else word for word in text.split()])

    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stopwords.words('english')]

    return ' '.join(tokens)

# Tokenize the text in df_train into sentences
sentences_train = [sent_tokenize(text) for text in df_train['text']]
sentences_train = [sentence for sublist in sentences_train for sentence in sublist]  # Flatten the list

# Apply the pre-processing function to each sentence
processed_sentences = [preprocess_text(sentence) for sentence in sentences_train]

processed_sentences[:5]  # Display first 5 processed sentences


['time rating continuously cast semis means hybrid quality models efficient reliable time determination surface internal quality continuous cast products deliver considerable contribution cost reduction efficiency improvement steel plants',
 'currently economic losses environmental impacts caused downgrading slabs billets additionally due conditioning increased material energy consumption occur',
 'nowadays development quality prediction models supported availability automatic inspection systems eg',
 'surface inspection hot cold rolled strips bars',
 'developing validating quality model concerning defects occur downstream process steps casting information inspection system reliable human decisions model adjusted accuracy']

In [ ]:
print(len(processed_sentences))

24432


In [ ]:
# Tokenize the text in df_train into sentences
sentences_test = [sent_tokenize(text) for text in df_test['text']]
sentences_test = [sentence for sublist in sentences_test for sentence in sublist]  # Flatten the list

# Apply the pre-processing function to each sentence
processed_sentences_test = [preprocess_text(sentence) for sentence in sentences_test]

processed_sentences_test[:5]

['simulated annealing metaheuristic solve optimal power flow optimal power flow problem widely studied order improve power systems operation planning',
 'real power systems problem formulated non linear large combinatorial problem',
 'first approaches used solve problem based mathematical methods required huge computational efforts',
 'lately artificial intelligence techniques metaheuristics based biological processes adopted',
 'metaheuristics require lower computational resources clear advantage addressing problem large power systems']

In [ ]:
print(len(processed_sentences_test))

8267


# TF-IDF weighted word2vec

In [ ]:
import os
import random
import re
from collections import Counter, OrderedDict
from dataclasses import dataclass
from time import monotonic
from typing import Dict, List, Optional, Union

import numpy as np
import torch
import torch.nn as nn
from scipy.spatial.distance import cosine
from torch.utils.data import DataLoader
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import WikiText103
from tqdm import tqdm

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

def get_data(train_sentences, valid_sentences):
    train_dataset = MyDataset(train_sentences)
    valid_dataset = MyDataset(valid_sentences)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

    return train_loader, valid_loader

# Get data loaders
train_loader, valid_loader = get_data(processed_sentences, processed_sentences_test)


In [ ]:
@dataclass
class Word2VecParams:

    # skipgram parameters
    MIN_FREQ = 50
    SKIPGRAM_N_WORDS = 8
    T = 85
    NEG_SAMPLES = 50
    NS_ARRAY_LEN = 5_000_000
    SPECIALS = ""
    TOKENIZER = 'basic_english'

    # network parameters
    BATCH_SIZE = 100
    EMBED_DIM = 300
    EMBED_MAX_NORM = None
    N_EPOCHS = 5
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    CRITERION = nn.BCEWithLogitsLoss()

In [ ]:
@dataclass
class Word2VecParams:

    # skipgram parameters
    MIN_FREQ = 50
    SKIPGRAM_N_WORDS = 8
    T = 85
    NEG_SAMPLES = 50
    NS_ARRAY_LEN = 5_000_000
    SPECIALS = ""
    TOKENIZER = 'basic_english'

    # network parameters
    BATCH_SIZE = 100
    EMBED_DIM = 300
    EMBED_MAX_NORM = None
    N_EPOCHS = 5
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    CRITERION = nn.BCEWithLogitsLoss()

In [ ]:
class Vocab:
    def __init__(self, list, specials):
        self.stoi = {v[0]:(k, v[1]) for k, v in enumerate(list)}
        self.itos = {k:(v[0], v[1]) for k, v in enumerate(list)}
        self._specials = specials[0]
        self.total_tokens = np.nansum(
            [f for _, (_, f) in self.stoi.items()]
            , dtype=int)

    def __len__(self):
        return len(self.stoi) - 1

    def get_index(self, word: Union[str, List]):
        if isinstance(word, str):
            if word in self.stoi:
                return self.stoi.get(word)[0]
            else:
                return self.stoi.get(self._specials)[0]
        elif isinstance(word, list):
            res = []
            for w in word:
                if w in self.stoi:
                    res.append(self.stoi.get(w)[0])
                else:
                    res.append(self.stoi.get(self._specials)[0])
            return res
        else:
            raise ValueError(
                f"Word {word} is not a string or a list of strings."
                )


    def get_freq(self, word: Union[str, List]):
        if isinstance(word, str):
            if word in self.stoi:
                return self.stoi.get(word)[1]
            else:
                return self.stoi.get(self._specials)[1]
        elif isinstance(word, list):
            res = []
            for w in word:
                if w in self.stoi:
                    res.append(self.stoi.get(w)[1])
                else:
                    res.append(self.stoi.get(self._specials)[1])
            return res
        else:
            raise ValueError(
                f"Word {word} is not a string or a list of strings."
                )


    def lookup_token(self, token: Union[int, List]):
        if isinstance(token, (int, np.int64)):
            if token in self.itos:
                return self.itos.get(token)[0]
            else:
                raise ValueError(f"Token {token} not in vocabulary")
        elif isinstance(token, list):
            res = []
            for t in token:
                if t in self.itos:
                    res.append(self.itos.get(token)[0])
                else:
                    raise ValueError(f"Token {t} is not a valid index.")
            return res


In [ ]:
def yield_tokens(iterator, tokenizer):
    r = re.compile('[a-z1-9]')
    for text in iterator:
        text = ' '.join(text) if isinstance(text, list) else text
        res = tokenizer(text)
        res = list(filter(r.match, res))
        yield res

def vocab(ordered_dict: Dict, min_freq: int = 1, specials: str = ''):
    tokens = []
    # Save room for special tokens
    for token, freq in ordered_dict.items():
        if freq >= min_freq:
            tokens.append((token, freq))

    specials = (specials, np.nan)
    tokens[0] = specials

    return Vocab(tokens, specials)

def pipeline(word, vocab, tokenizer):
    return vocab(tokenizer(word))

def build_vocab(
        iterator,
        tokenizer,
        params: Word2VecParams,
        max_tokens: Optional[int] = None,
    ):
    counter = Counter()
    for tokens in yield_tokens(iterator, tokenizer):
        counter.update(tokens)

    # First sort by descending frequency, then lexicographically
    sorted_by_freq_tuples = sorted(
        counter.items(), key=lambda x: (-x[1], x[0])
        )

    ordered_dict = OrderedDict(sorted_by_freq_tuples)

    word_vocab = vocab(
        ordered_dict, min_freq=params.MIN_FREQ, specials=params.SPECIALS
        )
    return word_vocab


In [ ]:
class SkipGrams:
    def __init__(self, vocab: Vocab, params: Word2VecParams, tokenizer):
        self.vocab = vocab
        self.params = params
        self.t = self._t()
        self.tokenizer = tokenizer
        self.discard_probs = self._create_discard_dict()

    def _t(self):
        freq_list = []
        for _, (_, freq) in list(self.vocab.stoi.items())[1:]:
            freq_list.append(freq/self.vocab.total_tokens)
        return np.percentile(freq_list, self.params.T)


    def _create_discard_dict(self):
        discard_dict = {}
        for _, (word, freq) in self.vocab.stoi.items():
            dicard_prob = 1-np.sqrt(
                self.t / (freq/self.vocab.total_tokens + self.t))
            discard_dict[word] = dicard_prob
        return discard_dict


    def collate_skipgram(self, batch):
        batch_input, batch_output  = [], []
        for text in batch:
            text_tokens = self.vocab.get_index(self.tokenizer(text))

            if len(text_tokens) < self.params.SKIPGRAM_N_WORDS * 2 + 1:
                continue

            for idx in range(len(text_tokens) - self.params.SKIPGRAM_N_WORDS*2
                ):
                token_id_sequence = text_tokens[
                    idx : (idx + self.params.SKIPGRAM_N_WORDS * 2 + 1)
                    ]
                input_ = token_id_sequence.pop(self.params.SKIPGRAM_N_WORDS)
                outputs = token_id_sequence

                prb = random.random()
                del_pair = self.discard_probs.get(input_)
                if input_==0 or del_pair >= prb:
                    continue
                else:
                    for output in outputs:
                        prb = random.random()
                        del_pair = self.discard_probs.get(output)
                        if output==0 or del_pair >= prb:
                            continue
                        else:
                            batch_input.append(input_)
                            batch_output.append(output)

        batch_input = torch.tensor(batch_input, dtype=torch.long)
        batch_output = torch.tensor(batch_output, dtype=torch.long)

        return batch_input, batch_output

In [ ]:
class NegativeSampler:
    def __init__(self, vocab: Vocab, ns_exponent: float, ns_array_len: int):
        self.vocab = vocab
        self.ns_exponent = ns_exponent
        self.ns_array_len = ns_array_len
        self.ns_array = self._create_negative_sampling()

    def __len__(self):
        return len(self.ns_array)

    def _create_negative_sampling(self):

        frequency_dict = {word:freq**(self.ns_exponent) \
                          for _,(word, freq) in
                          list(self.vocab.stoi.items())[1:]}
        frequency_dict_scaled = {
            word:
            max(1,int((freq/self.vocab.total_tokens)*self.ns_array_len))
            for word, freq in frequency_dict.items()
            }
        ns_array = []
        for word, freq in tqdm(frequency_dict_scaled.items()):
            ns_array = ns_array + [word]*freq
        return ns_array

    def sample(self,n_batches: int=1, n_samples: int=1):
        samples = []
        for _ in range(n_batches):
            samples.append(random.sample(self.ns_array, n_samples))
        samples = torch.as_tensor(np.array(samples))
        return samples

In [ ]:
class Model(nn.Module):
    def __init__(self, vocab: Vocab, params: Word2VecParams):
        super().__init__()
        self.vocab = vocab
        self.t_embeddings = nn.Embedding(
            self.vocab.__len__()+1,
            params.EMBED_DIM,
            max_norm=params.EMBED_MAX_NORM
            )
        self.c_embeddings = nn.Embedding(
            self.vocab.__len__()+1,
            params.EMBED_DIM,
            max_norm=params.EMBED_MAX_NORM
            )

    def forward(self, inputs, context):
        # getting embeddings for target & reshaping
        target_embeddings = self.t_embeddings(inputs)
        n_examples = target_embeddings.shape[0]
        n_dimensions = target_embeddings.shape[1]
        target_embeddings = target_embeddings.view(n_examples, 1, n_dimensions)

        # get embeddings for context labels & reshaping
        # Allows us to do a bunch of matrix multiplications
        context_embeddings = self.c_embeddings(context)
        # * This transposes each batch
        context_embeddings = context_embeddings.permute(0,2,1)

        # * custom linear layer
        dots = target_embeddings.bmm(context_embeddings)
        dots = dots.view(dots.shape[0], dots.shape[2])
        return dots

    def normalize_embeddings(self):
        embeddings = list(self.t_embeddings.parameters())[0]
        embeddings = embeddings.cpu().detach().numpy()
        norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
        norms = norms.reshape(norms.shape[0], 1)
        return embeddings / norms

    def get_similar_words(self, word, n):
        word_id = self.vocab.get_index(word)
        if word_id == 0:
            print("Out of vocabulary word")
            return

        embedding_norms = self.normalize_embeddings()
        word_vec = embedding_norms[word_id]
        word_vec = np.reshape(word_vec, (word_vec.shape[0], 1))
        dists = np.matmul(embedding_norms, word_vec).flatten()
        topN_ids = np.argsort(-dists)[1 : n + 1]

        topN_dict = {}
        for sim_word_id in topN_ids:
            sim_word = self.vocab.lookup_token(sim_word_id)
            topN_dict[sim_word] = dists[sim_word_id]
        return topN_dict

    def get_similarity(self, word1, word2):
        idx1 = self.vocab.get_index(word1)
        idx2 = self.vocab.get_index(word2)
        if idx1 == 0 or idx2 == 0:
            print("One or both words are out of vocabulary")
            return

        embedding_norms = self.normalize_embeddings()
        word1_vec, word2_vec = embedding_norms[idx1], embedding_norms[idx2]

        return cosine(word1_vec, word2_vec)

In [ ]:
class Trainer:
    def __init__(self, model: Model, params: Word2VecParams, optimizer,
                vocab: Vocab, train_iter, valid_iter, skipgrams: SkipGrams):
        self.model = model
        self.optimizer = optimizer
        self.vocab = vocab
        self.train_iter = train_iter
        self.valid_iter = valid_iter
        self.skipgrams = skipgrams
        self.params = params

        self.epoch_train_mins = {}
        self.loss = {"train": [], "valid": []}

        # sending all to device
        self.model.to(self.params.DEVICE)
        self.params.CRITERION.to(self.params.DEVICE)

        self.negative_sampler = NegativeSampler(
            vocab=self.vocab, ns_exponent=.75,
            ns_array_len=self.params.NS_ARRAY_LEN
            )
        self.testwords = ['learning', 'computer', 'machine', 'vision', 'language', 'network', 'technology', 'science', 'internet']


    # def train(self):
    #     self.test_testwords()
    #     for epoch in range(self.params.N_EPOCHS):
    #         # Generate Dataloaders
    #         self.train_dataloader = DataLoader(
    #             self.train_iter,
    #             batch_size=self.params.BATCH_SIZE,
    #             shuffle=False,
    #             collate_fn=self.skipgrams.collate_skipgram
    #         )
    #         self.valid_dataloader = DataLoader(
    #             self.valid_iter,
    #             batch_size=self.params.BATCH_SIZE,
    #             shuffle=False,
    #             collate_fn=self.skipgrams.collate_skipgram
    #         )
    #         # training the model
    #         st_time = monotonic()
    #         self._train_epoch()
    #         self.epoch_train_mins[epoch] = round((monotonic()-st_time)/60, 1)

    #         # validating the model
    #         self._validate_epoch()
    #         print(f"""Epoch: {epoch+1}/{self.params.N_EPOCHS}\n""",
    #         f"""    Train Loss: {self.loss['train'][-1]:.2}\n""",
    #         f"""    Valid Loss: {self.loss['valid'][-1]:.2}\n""",
    #         f"""    Training Time (mins): {self.epoch_train_mins.get(epoch)}"""
    #         """\n"""
    #         )
    #         self.test_testwords()

    def train(self):
        try:
            self.test_testwords()
            for epoch in range(self.params.N_EPOCHS):
                # Directly use self.train_iter and self.valid_iter
                self.train_dataloader = self.train_iter
                self.valid_dataloader = self.valid_iter

                # training the model
                st_time = monotonic()
                self._train_epoch()
                self.epoch_train_mins[epoch] = round((monotonic()-st_time)/60, 1)

                # validating the model
                self._validate_epoch()
                print(f"""Epoch: {epoch+1}/{self.params.N_EPOCHS}\n""",
                f"""    Train Loss: {self.loss['train'][-1]:.2}\n""",
                f"""    Valid Loss: {self.loss['valid'][-1]:.2}\n""",
                f"""    Training Time (mins): {self.epoch_train_mins.get(epoch)}"""
                """\n"""
                )
                self.test_testwords()

        except Exception as e:
            print(f"Done Execution")


    def _train_epoch(self):
        self.model.train()
        running_loss = []

        for i, batch_data in enumerate(self.train_dataloader, 1):
            if len(batch_data[0]) == 0:
                continue
            inputs = batch_data[0].to(self.params.DEVICE)
            pos_labels = batch_data[1].to(self.params.DEVICE)
            neg_labels = self.negative_sampler.sample(
                pos_labels.shape[0], self.params.NEG_SAMPLES
                )
            neg_labels = neg_labels.to(self.params.DEVICE)
            context = torch.cat(
                [pos_labels.view(pos_labels.shape[0], 1),
                neg_labels], dim=1
              )

            # building the targets tensor
            y_pos = torch.ones((pos_labels.shape[0], 1))
            y_neg = torch.zeros((neg_labels.shape[0], neg_labels.shape[1]))
            y = torch.cat([y_pos, y_neg], dim=1).to(self.params.DEVICE)

            self.optimizer.zero_grad()

            outputs = self.model(inputs, context)
            loss = self.params.CRITERION(outputs, y)
            loss.backward()
            self.optimizer.step()

            running_loss.append(loss.item())

        epoch_loss = np.mean(running_loss)

        self.loss['train'].append(epoch_loss)

    def _validate_epoch(self):
        self.model.eval()
        running_loss = []

        with torch.no_grad():
            for i, batch_data in enumerate(self.valid_dataloader, 1):
                if len(batch_data[0]) == 0:
                    continue
                inputs = batch_data[0].to(self.params.DEVICE)
                pos_labels = batch_data[1].to(self.params.DEVICE)
                neg_labels = self.negative_sampler.sample(
                    pos_labels.shape[0], self.params.NEG_SAMPLES
                    ).to(self.params.DEVICE)
                context = torch.cat(
                    [pos_labels.view(pos_labels.shape[0], 1),
                    neg_labels], dim=1
                  )


                # building the targets tensor
                y_pos = torch.ones((pos_labels.shape[0], 1))
                y_neg = torch.zeros((neg_labels.shape[0], neg_labels.shape[1]))
                y = torch.cat([y_pos, y_neg], dim=1).to(self.params.DEVICE)

                preds = self.model(inputs, context).to(self.params.DEVICE)
                loss = self.params.CRITERION(preds, y)

                running_loss.append(loss.item())

            epoch_loss = np.mean(running_loss)
            self.loss['valid'].append(epoch_loss)

    def test_testwords(self, n: int = 5):
        for word in self.testwords:
            print(word)
            nn_words = self.model.get_similar_words(word, n)
            for w, sim in nn_words.items():
                print(f"{w} ({sim:.3})", end=' ')
            print('\n')

In [ ]:
params = Word2VecParams()
train_iter, valid_iter = get_data(processed_sentences, processed_sentences_test)
tokenizer = get_tokenizer(params.TOKENIZER)
vocab = build_vocab(train_iter, tokenizer, params)
skip_gram = SkipGrams(vocab=vocab, params=params, tokenizer=tokenizer)
model = Model(vocab=vocab, params=params).to(params.DEVICE)
optimizer = torch.optim.Adam(params = model.parameters())

In [ ]:
for batch in train_iter:
    print(batch)
    break  # Only print the first batch

['combination spatial separable convolution mainly used enable amispp perform convolution regions different aspect ratios', 'information contains kind value', 'modern scenarios often deal massive high dimensional complex data spawned multiple sources natural consider ad perspective multiview deep learning', 'based byzantine fault tolerant bft mechanism paper improved anomaly detection method sdn multi controller fault tolerant abnormal wrong instructions ensured flow table issued correctly', 'using electro spinning method pvdf based nanofiber coated coper electrodes', 'mobile young people five million people world possessing great abilities', 'obtained mouse behavior data consist outliers behavioral variability addressed peirce criterion weighted least square regression wlsr respectively', 'sdn architectures decouples network control forwarding functions making network control become directly programmable underlying infrastructure abstracted applications network services', 'chapter two

In [ ]:
trainer = Trainer(
        model=model,
        params=params,
        optimizer=optimizer,
        train_iter=train_iter,
        valid_iter=valid_iter,
        vocab=vocab,
        skipgrams=skip_gram
    )
trainer.train()

100%|██████████| 1514/1514 [00:08<00:00, 176.18it/s]


learning
price (0.196) primary (0.196) account (0.181) might (0.158) surface (0.157) 

computer
classical (0.164) visualization (0.155) position (0.155) optical (0.153) mathematical (0.151) 

machine
analytics (0.203) features (0.2) likely (0.171) difference (0.153) caused (0.149) 

vision
tool (0.177) failure (0.171) day (0.167) dynamic (0.164) independent (0.158) 

language
comprehensive (0.212) range (0.199) essential (0.185) introducing (0.183) face (0.169) 

network
fuzzy (0.205) internet (0.182) public (0.166) share (0.166) center (0.165) 

technology
quantum (0.194) well (0.177) objective (0.175) services (0.174) guide (0.172) 

science
needs (0.188) made (0.187) twelve (0.174) day (0.156) expert (0.154) 

internet
either (0.187) network (0.182) opportunities (0.168) price (0.166) features (0.166) 

Done Execution


* Above the embeddings for the most similar words that shows the the testing of this model.